In [1]:
# load important modules

import numpy
import nltk
import sklearn
!pip install ipdb
import ipdb
!pip install tensorboardX
import tensorboardX
import pytorch as torch

In [2]:
# Clone GitHub repository to enable downloading GLUE data
!git clone https://github.com/nyu-mll/GLUE-baselines
!git clone https://github.com/wasiahmad/paraphrase_identification.git

fatal: destination path 'GLUE-baselines' already exists and is not an empty directory.
fatal: destination path 'paraphrase_identification' already exists and is not an empty directory.


In [7]:
# Download GLUE data
!python GLUE-baselines/download_glue_data.py --data_dir="C:/Users/annko/Documents/Masterthesis/Masterthesis" --tasks all --path_to_mrpc=paraphrase_identification/dataset/msr-paraphrase-corpus

	Completed!
	Completed!
Processing MRPC...
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from collections import defaultdict
import math

In [9]:
# define data direction to read data
data_dir = "C:/Users/annko/Documents/Masterthesis/Masterthesis/"

# read respective GLUE tasks

# WNLI
wnli_train = pd.read_csv("{}/WNLI/train.tsv".format(data_dir), delimiter='\\t')
wnli_dev = pd.read_csv("{}/WNLI/dev.tsv".format(data_dir), delimiter='\\t')

# RTE
rte_train = pd.read_csv("{}/RTE/train.tsv".format(data_dir), delimiter='\\t')
rte_dev = pd.read_csv("{}/RTE/dev.tsv".format(data_dir), delimiter='\\t')

# MRPC
mrpc_train = pd.read_csv("{}/MRPC/train.tsv".format(data_dir), delimiter='\\t', header=None, skiprows=1, 
                    names=['Quality', 'ID1', 'ID2', 'String2', 'String'])
mrpc_dev = pd.read_csv("{}/MRPC/dev.tsv".format(data_dir), delimiter='\\t', header=None, skiprows=1, 
                  names=['Quality', 'ID1', 'ID2', 'String2', 'String'])

# STS-B
stsb_train = pd.read_csv("{}/STS-B/train.tsv".format(data_dir), delimiter='\\t')
stsb_dev = pd.read_csv("{}/STS-B/dev.tsv".format(data_dir), delimiter='\\t')

# CoLA
cola_train = pd.read_csv("{}/CoLA/train.tsv".format(data_dir), delimiter='\t', header=None, skiprows=1, 
                    names=['sentence_source', 'label', 'label_notes', 'sentence'])
cola_dev = pd.read_csv("{}/CoLA/dev.tsv".format(data_dir), delimiter='\t', header=None, skiprows=1, 
                  names=['sentence_source', 'label', 'label_notes', 'sentence'])

# SST-2
sst2_train = pd.read_csv("{}/SST-2/train.tsv".format(data_dir), delimiter='\t')
sst2_dev = pd.read_csv("{}/SST-2/dev.tsv".format(data_dir), delimiter='\t')

# QQP
qqp_train = pd.read_csv("{}/QQP/train.tsv".format(data_dir), delimiter='\\t', quotechar='"')
qqp_dev = pd.read_csv("{}/QQP/dev.tsv".format(data_dir), delimiter='\\t', quotechar='"')

# QNLI
qnli_train = pd.read_csv("{}/QNLI/train.tsv".format(data_dir), delimiter='\\t')
qnli_dev = pd.read_csv("{}/QNLI/dev.tsv".format(data_dir), delimiter='\\t')

# MNLI
mnli_train = pd.read_csv("{}/MNLI/train.tsv".format(data_dir), delimiter='\\t')
mnli_dev_m = pd.read_csv("{}/MNLI/dev_matched.tsv".format(data_dir), delimiter='\\t')
mnli_dev_mism = pd.read_csv("{}/MNLI/dev_mismatched.tsv".format(data_dir), delimiter='\\t')

<ipython-input-9-1d609790f022>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  wnli_train = pd.read_csv("{}/WNLI/train.tsv".format(data_dir), delimiter='\\t')
<ipython-input-9-1d609790f022>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  wnli_dev = pd.read_csv("{}/WNLI/dev.tsv".format(data_dir), delimiter='\\t')
<ipython-input-9-1d609790f022>:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  rte_train = pd.rea

In [ ]:
# have a look at data samples
datalist = [wnli_train, rte_train, mrpc_train, stsb_train, cola_train, sst2_train, qqp_train, qnli_train, mnli_train]
data_names = ['WNLI', 'RTE', 'MRPC', 'STS-B', 'CoLA', 'SST-2', 'QQP', 'QNLI', 'MNLI']

pd.set_option('display.max_colwidth', -1) # to enable viewing whole sentences

counter = 0
for data in datalist:
    print(data_names[counter])
    print(data.sample(3))
    print('\n')
    counter += 1

In [ ]:
# regard info of respective GLUE tasks
counter = 0
for data in datalist:
    print(data_names[counter])
    print(data.info())
    print('\n')
    counter += 1
    
# WNLI, MRPC, STS-B, SST-2, CoLA, QQP and QNLI contain numeric labels; RTE and MNLI contains labels with categoric values

In [ ]:
# distingish respective label names
counter = 0
for i in datalist:
    print(data_names[counter])
    print(i.columns)
    print('\n')
    counter += 1

In [ ]:
def label_counts_cat(data, data_label, dataname):
    print('{}'.format(dataname))
    print('Number of sentences: {:,}'.format(len(data)))
    print(data_label.value_counts(normalize=True))
    print('\n')

In [ ]:
# print number of sentences and value counts of respective labels
label_list = [wnli_train.label, rte_train.label, mrpc_train.Quality, stsb_train.score, cola_train.label, sst2_train.label, 
             qqp_train.is_duplicate, qnli_train.label, mnli_train.gold_label]

counter = 0
for i in datalist:
    label_counts_cat(i, label_list[counter], data_names[counter])
    counter += 1

In [5]:
### Histogram for STS-B

plt.figure(figsize = (10, 8))
n, bins, patches = plt.hist(x=stsb_train.score, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value', fontsize = 16)
plt.ylabel('Frequency', fontsize = 16)
# plt.title('Distribution of STS-B scores', fontsize = 18)
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
plt.setp(ax.get_xticklabels(), rotation='horizontal', fontsize=16)
plt.setp(ax.get_yticklabels(), rotation='horizontal', fontsize=16)

plt.savefig('stsb-hist.png')

NameError: name 'stsb_train' is not defined

<Figure size 720x576 with 0 Axes>

In [26]:
# print number of examples for development sets
dev_list = [wnli_dev, rte_dev, mrpc_dev, cola_dev, stsb_dev, sst2_dev, qqp_dev, qnli_dev, mnli_dev_m, mnli_dev_mism]
dev_names = ['WNLI', 'RTE', 'MRPC', 'STS-B', 'CoLA', 'SST-2', 'QQP', 'QNLI', 'MNLI_matched', 'MNLI_mismatched']

counter = 0
for dev in dev_list:
    print('{}'.format(dev_names[counter]))
    print('Number of sentences: {:,}'.format(dev.shape[0]))
    print('\n')
    counter += 1

WNLI
Number of sentences: 71


RTE
Number of sentences: 277


MRPC
Number of sentences: 408


STS-B
Number of sentences: 1,042


CoLA
Number of sentences: 1,500


SST-2
Number of sentences: 872


QQP
Number of sentences: 40,431


QNLI
Number of sentences: 5,732


MNLI_matched
Number of sentences: 9,815


MNLI_mismatched
Number of sentences: 9,832


